In [1]:
from bloom_filter import BloomFilter
from random import shuffle

Size of bit array: 124
False positive Probability: 0.05 %
Number of hash functions: 4
'nuke' is definitely not present!
'bolster' is probably present!
'gloomy' is definitely not present!
'cohesive' is probably present!
'bluff' is definitely not present!
'comely' is probably present!
'cheater' is definitely not present!
'twitter' is a false positive!
'generosity' is probably present!
'bonny' is probably present!
'colorful' is probably present!
'war' is definitely not present!
'geeksforgeeks' is definitely not present!
'comfort' is probably present!
'gems' is probably present!
'facebook' is definitely not present!
'bonuses' is probably present!
'abound' is probably present!
'humanity' is definitely not present!
'hurt' is definitely not present!
'racism' is definitely not present!
'hate' is definitely not present!


# Exam

## Statistics

### 13. Probabilistic Data Structures
A very interesting application of probability in computer science is a kind of data structures which have a probabilistic behaviour. Examples of these are **Bloom filter**, **Skip list**, **Count-min sketch** and **HyperLogLog**.

Research how one of these structures works. Or write about many of them, if you wish. You can use the following checklist as a guide: 
* ~~What is a data structure?~~
* ~~What is a probabilistic data structure?~~
    * Where does the probabilistic behaviour emerge?
    * ~~What advantages do these structures provide?~~
* ~~For your chosen structure, how is it constructed?~~
    * ~~What parts do you need? What are the details?~~
* ~~How does the structure work?~~
    * ~~What operations can you do?~~
    * What are the typical probabilities associated with these operations?
* Analyze the structure
    * Analyze the runtimes for all operations
    * Analyze the space usage
    * Compare to a similar, non-probabilistic data structure
    * What advantages does the new data structure have? What drawbacks do you need to be aware of?
* Give at least one example where this structure is useful
    * E.g. Bloom filter - spell checkers
    * Analyze the use case
    * If possible, implement the use case
    * Display some metrics (e.g. % conserved space, % reduced time)

### What is data structure?
In computer science, a data structure is a data organization, management, and storage format that enables efficient access and modification. More precisely, a data structure is a collection of data values, the relationships among them, and the functions or operations that can be applied to the data.

### What is a probabilistic data structure?
Probabilistic data structures are a group of data structures that are extremely useful for big data and streaming applications. Generally speaking, these data structures use hash functions to randomize and compactly represent a set of items. Collisions are ignored but errors can be well-controlled under certain threshold. Comparing with error-free approaches, these algorithms use much less memory and have constant query time. They usually support union and intersection operations and therefore can be easily parallelized.

#### What advantages do these structures provide?
Probabilistic data structures can't give you a definite answer, instead they provide you with a reasonable approximation of the answer and a way to approximate this estimation. They are extremely useful for big data and streaming application because they allow to dramatically decrease the amount of memory needed (in comparison to data structures that give you exact answers).

In majority of the cases these data structures use hash functions to randomize the items. Because they ignore collisions they keep the size constant, but this is also a reason why they can't give you exact values. The advantages they bring:

- they use small amount of memory (you can control how much)
- they can be easily parallelizable (hashes are independent)
- they have constant query time (not even amortized constant like in dictionary)

### Bloom Filter
The Bloom filter is a space-efficient probabilistic data structure that is used to test whether an element is a member of a set.  The price we pay for efficiency is that it is probabilistic in nature that means, there might be some False Positive results. False positive means, it might tell that given username is already taken but actually it’s not. Bloom filters are both fast and space-efficient. However, elements can only be added, not removed.

For example, checking availability of username is set membership problem, where the set is the list of all registered username.

Boom filter requires these three inputs:
* m: size of the bit array
* n: estimated insertion
* p: false positive probability

An empty Bloom filter is a bit array of m bits, all set to 0. There are also k different hash functions, each of which maps a set element to one of the m bit positions.
* To add an element, feed it to the hash functions to get k bit positions, and set the bits at these positions to 1.
* To test if an element is in the set, feed it to the hash functions to get k bit positions.
    * If any of the bits at these positions is 0, the element definitely isn’t the set.
    * If all are 1, then the element may be in the set.

For understanding bloom filters, you must know what is hashing.<br>
A hash function takes input and outputs a unique identifier of fixed length which is used for identification of input.<br> 
A hash function is a function that takes a set of inputs of any arbitrary size and fits them into a table or other data structure that contains fixed-size elements.

### How does the structure work?
First of all an empty bloom filter is a bit array of m bits, all set to zero, like that one:
<img src="Images/bloom_filter1.png">
We need k number of hash functions to calculate the hashes for a given input. When we want to add an item in the filter, the bits at k indices h1(x), h2(x), … hk(x) are set, where indices are calculated using hash functions. Example – Suppose we want to enter “geeks” in the filter, we are using 3 hash functions and a bit array of length 10, all set to 0 initially. First we’ll calculate the hashes as following :

<br><center>h1("geeks") % 10 = 1 <br>
h1("geeks") % 10 = 4 <br>
h1("geeks") % 10 = 7</center><br>

Now we are going to set the indexes on positions 1, 4 and 7 to 1.<br>
<img src="Images/bloom_filter2.png">

And if we want to enter a word, for example. First we will calculate the hashes.

<br><center>h1(“nerd”) % 10 = 3<br>
h2(“nerd”) % 10 = 5<br>
h3(“nerd”) % 10 = 4</center>

So we will set the indexes on positions 3, 5 and 4 to 1.<br>
<img src="Images/bloom_filter3.png">

Now if we want to check “geeks” is present in filter or not. We’ll do the same process but this time in reverse order. We calculate respective hashes using h1, h2 and h3 and check if all these indices are set to 1 in the bit array. If all the bits are set then we can say that “geeks” is probably present. If any of the bit at these indices are 0 then “geeks” is definitely not present.

But why do we called it probably present. Isn't it uncertain? Let's suppose we want to check whether “cat” is present or not. We’ll calculate hashes using h1, h2 and h3.

<center>h1(“cat”) % 10 = 1<br>
h2(“cat”) % 10 = 3<br>
h3(“cat”) % 10 = 7</center>

If we check the bit array, bits at these indices are set to 1 but we know that “cat” was never added to the filter. Bit at index 1 and 7 was set when we added “geeks” and bit 3 was set we added “nerd”.
<img src="Images/bloom_filter4.png">

So, because bits at calculated indices are already set by some other item, bloom filter erroneously claim that “cat” is present and generating a false positive result. Depending on the application, it could be huge downside or relatively okay.
We can control the probability of getting a false positive by controlling the size of the Bloom filter. More space means fewer false positives. If we want decrease probability of false positive result, we have to use more number of hash functions and larger bit array. This would add latency in addition of item and checking membership. 

#### What operations can be done?
Operations that a Bloom Filter supports  

* insert(x) : To insert an element in the Bloom Filter.
* lookup(x) : to check whether an element is already present in Bloom Filter with a positive false probability.

In [4]:
n = 20 # no of items to add
p = 0.05 # false positive probability

bloomf = BloomFilter(n,p)
print(f"Size of bit array: {bloomf.size}")
print(f"False positive Probability: {bloomf.fp_prob} %")
print(f"Number of hash functions: {bloomf.hash_count}\n")

# words to be added
word_present = ['abound','abounds','abundance','abundant','accessable',
				'bloom','blossom','bolster','bonny','bonus','bonuses',
				'coherent','cohesive','colorful','comely','comfort',
				'gems','generosity','generous','generously','genial']

# word_present = ['geeks', 'nerd', 'cat']

# word not added
word_absent = ['bluff','cheater','hate','war','humanity',
			'racism','hurt','nuke','gloomy','facebook',
			'geeksforgeeks','twitter']

for item in word_present:
	bloomf.add(item)

shuffle(word_present)
shuffle(word_absent)

test_words = word_present[:10] + word_absent
shuffle(test_words)
for word in test_words:
	if bloomf.check(word):
		if word in word_absent:
			print("'{}' is a false positive!".format(word))
		else:
			print("'{}' is probably present!".format(word))
	else:
		print("'{}' is definitely not present!".format(word))

Size of bit array: 124
False positive Probability: 0.05 %
Number of hash functions: 4

'abundance' is probably present!
'humanity' is definitely not present!
'nuke' is definitely not present!
'hurt' is definitely not present!
'generosity' is probably present!
'twitter' is a false positive!
'comely' is probably present!
'abound' is probably present!
'cohesive' is probably present!
'coherent' is probably present!
'abundant' is probably present!
'bluff' is definitely not present!
'bonuses' is probably present!
'comfort' is probably present!
'facebook' is definitely not present!
'generously' is probably present!
'cheater' is definitely not present!
'geeksforgeeks' is definitely not present!
'gloomy' is definitely not present!
'hate' is definitely not present!
'racism' is definitely not present!
'war' is definitely not present!
